# Trabalho final ML

<a href="https://www.kaggle.com/code/reihanenamdari/depression-logistic-regression-and-gridsearchcv/data">Dataset</a>
<br>
<hr>

<br>

#### Contexto e objetivo do trabalho
O trabalho prático da disciplina CMP263 - Aprendizagem de Máquina visa
permitir que os alunos desenvolvam um modelo preditivo para um problema de
interesse, praticando aspectos discutidos na disciplina relacionados ao treinamento e
avaliação de modelos de classificação ou regressão, e interpretação dos modelos
gerados.
A proposta do projeto final é que os alunos aprofundem e consolidem sua
experiência no desenvolvimento de modelos preditivos, abordando aspectos ao
longo de toda a metodologia de treinamento de modelos, conforme discutimos em
aula. 

## Metodologia

CRISP-DM

1. Entendimento do negócio
2. Entendimento dos dados
3. Preparação dos dados
4. Modelagem
5. Validação
6. Deployment

## 1. Entendimento do negócio

### About Dataset
##### Context


O dataset original foi publicado pro Frankcc no seguinte link: <a href="https://www.kaggle.com/datasets/diegobabativa/depression?resource=download">Link Kaggle</a>

O dataset está envolvido na análise da depressão. Os dados consistem de um estudo das condições de vida de pessoas que vivem em zonas rurais. 

- Conteúdo
1. Surveyid 
2. Villeid
3. sex
4. Age
5. Married
6. Numberchildren 
7. educationlevel
8. totalmembers (in the family) 
9. gainedasset
10. durableasset 
11. saveasset
12. livingexpenses 
13. otherexpenses
14. incomingsalary 
15. incomingownfarm 
16. incomingbusiness
17. incomingnobusiness
18. incomingagricultural 
19. farmexpenses
20. laborprimary 
21. lastinginvestment
22. nolastinginvestmen
23. depressed: [ Zero: No depressed] or [One: depressed] (Binary for target class)

the main objective is to show statistic analysis and some data mining techniques.

The dataset has 23 columns or dimensions and a total of 1432 rows or objects.

Acknowledgements
The original attribution is to Frankcc i

Inspiration

<a href="https://zindi.africa/competitions/busara-mental-health-prediction-challenge/data">Busara</a>

## Problema

Classificação -  objetivo é treinar o melhor classificador possível para detectar o risco de depressão, pois os dados da depressão são categóricos (0,1). Induz um classificador, gerando saídas em um domínio discreto, não ordenado 

yk = f (xk) ∈ {c1, c2, …, cm},
onde m é o número de classes.
Para m = 2: classificação binária
Para m > 2: classificação multiclasse

- Classificação binária

Pergunta:

- 1. Dada as características socioeconômicas do indivíduo no survey, é possível classificar de forma assertiva se este indivíduo é depressivo ou não?


Tirando esta parte, você precisa avaliar as necessidades de pré-processamento dos dados, como imputar valor faltantes, tratar outliers, normalizar, balancear as classes, etc... Você pode aplicar seleção de atributos se deseja avaliar se um subconjunto dos atributos originais já lhe dar bom poder preditivo.

Esse pipeline será integrado a algoritmos de classificação (veja bem, suas saídas são numéricas, mas não se trata de uma regressão. Você tem valores na classe 0 ou 1, pois as classes foram codificadas dessa forma) - e o objetivo será treinar o melhor classificador possível para detectar o risco de depressão.

 

## Configs de ambiente

<hr>
SO - Ubuntu 20.4
<br>
Python version - 3.7.6
<br>
Conda version version - 4.8.2


<hr>


In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import sequence
import seaborn as sns
import matplotlib.pyplot as plt
import os
from mpl_toolkits.mplot3d import Axes3D
from tqdm.notebook import trange, tqdm
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)
import random as r


%matplotlib inline

## i) análise exploratória dos dados
para identificar possíveis problemas nos
dados que possam impactar negativamente no treinamento de modelos;

In [ ]:
for dirname, _, filenames in os.walk('/home/kaline/workspace/mestrado/TrabalhoFinalML/Dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
target_column_name = 'depressed'

In [ ]:
df_depressed_dataset = pd.read_csv("Dataset/depressed.csv")

# Create a boxplot of life expectancy per region
df_depressed_dataset.boxplot('depressed')

# Show the plot
plt.show()


In [ ]:
df_depressed_dataset.head()

### Distribuição da coluna target (depressed)

Classes não balanceadas

In [ ]:
#Contagem de registros por classes
from turtle import color


target_counts = df_depressed_dataset['depressed'].value_counts()
positives = target_counts[1]
negatives = target_counts[0]

#Proporção em %
prop = (positives/negatives)*100
print('\nA variável target do treino possui {}% de positivos.'. format(round(prop,2)))

#Gráfico
plt.bar(target_counts.index, target_counts,color='red')
plt.xticks([0, 1])
plt.xlabel('Classe')
plt.ylabel('Quantidade')
plt.title('Quantidade de registros por classe')


#### Tipos de dados nas colunas

In [ ]:
tipos_dados = df_depressed_dataset.dtypes.value_counts()

#Recurso visual
plt.bar(tipos_dados.index.astype(str), tipos_dados.values, color='red')
plt.ylabel('Quantidade')
plt.title('Quantidade de colunas por tipos de dados')

In [ ]:
print('Podemos observar que temos variáveis em diferentes escalas\nO desvio padrão (std) alto')
df_depressed_dataset.describe()

### Count NAN in each column

In [ ]:
df_depressed_dataset.isnull().sum().sort_values().tail(30)

## Detecção de outliers

Melhorar a perfomance do R² (score)

In [ ]:
sns.boxplot(df_depressed_dataset.children)

In [ ]:
sns.boxplot(df_depressed_dataset.edu)

In [ ]:
sns.boxplot(df_depressed_dataset.hh_children)

In [ ]:
df_depressed_dataset

In [ ]:
df_depressed_dataset.head(5)

In [ ]:

# correlation 
print(df_depressed_dataset.corr())

## ii) pré-processamento dos dados
abordando aspectos como correção de
outliers e de valores faltantes, codificação de atributos categóricos, discretização de
atributos numéricos, normalização, ajuste de desbalanceamento de classes e
redução de dimensionalidade

#### Removendo colunas irrelevantes

Minha sugestão no primeiro momento é remover apenas as colunas que pela sua definição não parecem ter relevância/poder preditivo, o que eu acredito ser o caso das duas variáveis com "_id" no final (não olhei em profundidade os dados). Se as variáveis forem categóricas e tiverem muitos valores possíveis, também pode ser difícil usar no modelo. Neste caso, você pode optar por manter apenas um subconjunto de valores para as categorias mais frequentes, e todas as outras agrupar em uma única categoria ("Outros", por exemplo).

In [ ]:
# Colunas categoricas
print(df_depressed_dataset.select_dtypes(include=['object']).columns.tolist())

# Print shape do DataFrame original
print("Shape do DataFrame original: {}".format(df_depressed_dataset.shape))

# Não tenho tempo para trabalhar com colunas de data agora
df_depressed_dataset.drop(columns=['surveyid','village','survey_date','hhsize','cons_nondurable',
'asset_livestock','asset_durable','asset_phone','asset_land_owned_total',
'asset_niceroof','cons_allfood','cons_ownfood','cons_med_total',
'cons_med_children','cons_ed','cons_social','cons_other','ent_wagelabor',
'ent_ownfarm','ent_business','ent_nonagbusiness','ent_employees',
'ent_nonag_revenue','ent_nonag_flowcost','ent_farmrevenue','ent_farmexpenses',
 'ent_animalstockrev','ent_total_cost','fs_adskipm_often','fs_adwholed_often',
 'fs_chskipm_often','fs_chwholed_often','fs_meat','fs_enoughtom','fs_sleephun',
 'med_expenses_hh_ep','med_expenses_sp_ep','med_expenses_child_ep',
 'med_portion_sickinjured','med_port_sick_child','med_afford_port','med_sickdays_hhave',
 'med_healthconsult','med_vacc_newborns','med_child_check','med_u5_deaths','ed_expenses',
 'ed_expenses_perkid','ed_schoolattend','ed_sch_missedpc',
 'ed_work_act_pc','labor_primary','wage_expenditures','early_survey','day_of_week'], axis=1, inplace=True)


# Print shape do novo DataFrame
print("Shape of DataFrame depois as colunas irrelevantes: {}".format(df_depressed_dataset.shape))
# durable_investment,nondurable_investment,given_mpesa,amount_given_mpesa,received_mpesa,amount_received_mpesa,net_mpesa,saved_mpesa,amount_saved_mpesa



In [ ]:
df_depressed_dataset.columns
df_depressed_dataset.hist(figsize=[20, 15])

### Removendo NAN


In [ ]:
df_depressed_dataset.columns

In [ ]:
df_depressed_dataset

In [ ]:
df_columns = ['depressed']

for df_column in df_columns:
   print(df_column)
   drop_nan = df_depressed_dataset.drop(df_depressed_dataset[(df_depressed_dataset[df_column].isnull())].index, axis=0, inplace=True)

df_filter = df_depressed_dataset.isnull().sum()

df_filter

In [ ]:
#Imputing dados faltantes em uma ML Pipeline I
# Import o módulo KNNImputer 
from sklearn.impute import KNNImputer 

# Setup the Imputation transformer: imputer
imputer = KNNImputer(n_neighbors=5)
columnsDropNan = df_depressed_dataset[['hh_totalmembers', 'cons_alcohol', 'cons_tobacco']] 
df_depressed_dataset[['hh_totalmembers', 'cons_alcohol', 'cons_tobacco']] = imputer.fit_transform(columnsDropNan.values) 
# Setup the pipeline with the required steps: steps
steps = [('imputation', imputer)]


In [ ]:
df_filter = df_depressed_dataset.isnull().sum()
df_filter

In [ ]:
df_depressed_dataset

#### Valores ausentes

In [ ]:
# Função que deleta registros duplicados da base (Mantendo apenas o primeiro registro a cada grupo de duplicatas)
def DupRegClean(df):
    # Registros duplicados podem causar ruído nos algoritmos de machine learning portanto iremos excluí-los.
    # (Se por acaso colunas duplicadas estierem presentes nos datasets de treino e teste, os resultados podem se mostrar tendenciosos).
    print('df antes: ',df.shape)
    df_saida = df.drop_duplicates(inplace=False,keep='first')
    print('Após a limpeza de registros duplicados: ',df_saida.shape)
    return df_saida

# Aplica limpeza
df1 = DupRegClean(df_depressed_dataset)

### Balancemento de classes

In [ ]:
from cmath import nan
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

X = df_depressed_dataset.drop(target_column_name, axis=1).fillna(0)
y = df_depressed_dataset[target_column_name]

# undersampling e oversampling

rus = RandomUnderSampler(random_state=0, sampling_strategy={0:600})
x_novo, y_novo = rus.fit_resample(X, y)
print(sorted(Counter(y_novo).items()))

df_depressed_dataset.iloc[:,:-1] =  x_novo
df_depressed_dataset[target_column_name] = y_novo
df_depressed_dataset = df_depressed_dataset.dropna()

rus = RandomOverSampler(random_state=0, sampling_strategy={1:194})
x_novo, y_novo = rus.fit_resample(x_novo, y_novo)
print(sorted(Counter(y_novo).items()))

df_depressed_dataset.iloc[:,:-1] =  x_novo
df_depressed_dataset[target_column_name] = y_novo
df_depressed_dataset = df_depressed_dataset.dropna()



In [ ]:
target_counts = df_depressed_dataset['depressed'].value_counts()
positives = target_counts[1]
negatives = target_counts[0]

#Proporção em %
prop = (positives/negatives)*100
print('\nA variável target do treino possui {}% de positivos.'. format(round(prop,2)))

#Gráfico
plt.bar(target_counts.index, target_counts,color='red')
plt.xticks([0, 1])
plt.xlabel('Classe')
plt.ylabel('Quantidade')
plt.title('Quantidade de registros por classe')

In [ ]:
conda install imbalanced-learn

In [ ]:
df_depressed_dataset

In [ ]:
! pip install -U imbalanced-learn


In [ ]:
df_depressed_dataset.head()

In [ ]:
df_depressed_dataset.columns

#### z-score - remover outliers

Z score = (x -mean) / std. deviation

- df antes:  (1409, 23)


In [ ]:
df_columns_z = ['children', 'edu', 'hh_children',
       'hh_totalmembers', 'asset_savings', 'cons_alcohol', 'cons_tobacco',
       'durable_investment', 'nondurable_investment', 'given_mpesa',
       'amount_given_mpesa', 'received_mpesa', 'amount_received_mpesa',
       'net_mpesa', 'saved_mpesa', 'amount_saved_mpesa']

for column in df_columns_z:
    print(column)
    m = np.mean(df_depressed_dataset[column])
    s = np.std(df_depressed_dataset[column])
    print('\nm\n', m, '\ns\n', s)
    df_depressed_dataset['Z-score'] = (df_depressed_dataset[column] - m)/s
    df_outlier = df_depressed_dataset[abs(df_depressed_dataset['Z-score']) > 3]
    df_depressed_dataset = df_depressed_dataset.drop(df_outlier.index)
    print('2 - \nm\n', m, '\ns\n', s)
    sns.boxplot(x=df_depressed_dataset.hh_children)

df_depressed_dataset.drop('Z-score', axis=1, inplace=True)

In [ ]:
sns.boxplot(df_depressed_dataset.children)

In [ ]:
df_depressed_dataset

### Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

# Setup the pipeline steps: steps
steps = [('imputation', imputer)]
        
# Create the pipeline: pipeline
pipeline = Pipeline(steps)

### Dividindo datasets

In [ ]:
# Separando as variáveis da target
X = df_depressed_dataset.drop(target_column_name, axis=1).fillna(0)
y = df_depressed_dataset[target_column_name]

# Dividindo o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42,stratify=y)

# Observando as proporções de classes nas targets do dataset de treino e teste (o objetivo é ter uma proporção semelhante)
fig,ax = plt.subplots(figsize=(8,6))
treino_b = ax.bar(y_train.value_counts().index-0.2,y_train.value_counts().values,0.4, color='gray',label='Dataset Treino')
teste_b  = ax.bar( y_test.value_counts().index+0.2,y_test.value_counts().values ,0.4, color='red' , label='Dataset Teste')

# Percentuais
perc_tr = (y_train.value_counts().values[1]/y_train.value_counts().values[0])*100
print('\nO dataset de treino possui: ',round(perc_tr,2),'% de classes positivas')
perc_ts = (y_test.value_counts().values[1]/y_test.value_counts().values[0])*100
print('\nO dataset de teste possui: ',round(perc_ts,2),'% de classes positivas')

# Recurso visual
ax.set_ylabel('Quantidade')
ax.set_xlabel('Target')
ax.set_title('Proporção de classes na variavel target')
ax.set_xticks([0,1])
ax.legend()
fig.tight_layout()
plt.show()
X_train

#### Padronização das features para aplicação do modelo

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
column_names_to_normalize = ['children', 'edu', 'hh_children',
       'hh_totalmembers', 'asset_savings', 'cons_alcohol', 'cons_tobacco',
       'durable_investment', 'nondurable_investment', 'given_mpesa',
       'amount_given_mpesa', 'received_mpesa', 'amount_received_mpesa',
       'net_mpesa', 'saved_mpesa', 'amount_saved_mpesa']
x = df_depressed_dataset[column_names_to_normalize].values
x_scaled =  scaler.fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = df_depressed_dataset.index)
df_depressed_dataset[column_names_to_normalize] = df_temp


""" # Padronização das features para aplicação do modelo
scaler = StandardScaler()

# Transformação
X_train_ = X_train.copy()
print("X_train_", X_train_)
X_test_ = X_train.copy()
X_train = scaler.fit_transform(X_train)
print("X_train ", X_train)

X_test  = scaler.transform(X_test)

# Variancia total do dataframe após a padronização
std_apos = X_train.std()
print('Desvio padrão após a padronização: ', round(std_apos,2))
# Transformando os datasets em pandas dataframes para maior facilidade de manipulação
X_train = pd.DataFrame(X_train)
X_test  = pd.DataFrame(X_test) """


#### Limpeza de dados

## iii) treinamento e validação dos modelos,
utilizando as melhores práticas em relação a estratégias de divisão de dados para otimização de hiperparâmetros e
seleção de modelos

 - Acurácia
 - Precision
 - Recall
 - F1 Score
 - ROC AUC score

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline


# Instancia-se o modelo burro
dc = DummyClassifier(strategy="most_frequent") # O Dummy classifier vai prever sempre a target mais frequente no dataset de treino

# Treino do modelo burro
dc.fit(X_train, y_train)

# Teste de performance
pred_prob = dc.predict_proba(X_train)
preds = dc.predict(X_train)
dumb_score = roc_auc_score(y_train, pred_prob[:,1])
duymb_acc = accuracy_score(y_train, preds)


print('\nAUC Score modelo dumb: ',dumb_score)
print('\nAccuracy Score modelo dumb: ',round(duymb_acc,2))

### Classificação 
Métodos baseados em instâncias
○ K-Vizinhos Mais Próximos
● Métodos baseados em procura
○ Árvores de decisão
● Métodos probabilísticos
○ Naïve Bayes
● Métodos estatísticos
○ Regressão Linear e Regressão Logística
● Métodos baseados em otimização
○ Redes Neurais Artificiais
● Métodos baseados em múltiplos modelos preditivos
○ Florestas Aleatórias, Adaboost, e outros

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
#from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import recall_score, precision_score, mean_squared_error, confusion_matrix

# Parâmetros iniciais para cada algoritmo (Vamos buscar ajustar inicialmente os 
# hiperparametros para trabalharem com a target desbalanceada)
naive_bayes_params = [{}]
#xgb_params         = [{"scale_pos_weight":5,"random_state":42}] # 100% de targets dividido pela quantidade 
                                                                # de positivos (20%) = 5
kneighbors_params  = [{"weights":"distance"}] 
svc_params         = [{"class_weight":"balanced", "random_state":42}]
log_reg_params     = [{"class_weight":"balanced","max_iter":10000, "random_state":42}]
dec_tree_params    = [{"class_weight":"balanced", "random_state":42}]
rand_for_params    = [{"class_weight":"balanced","random_state":42}]
mlp_for_params     = [{"activation":"identity", "random_state":42}]

modelclasses = [
     ["naive bayes",       GaussianNB,                 naive_bayes_params]
    #,["XGBoost",           XGBClassifier,                      xgb_params]
    ,["k neighbors",       KNeighborsClassifier,        kneighbors_params]
    ,["support vector classifier", LinearSVC,                  svc_params]
    ,["log regression",    LogisticRegression,             log_reg_params]
    ,["decision tree",     DecisionTreeClassifier,        dec_tree_params]
    ,["random forest",     RandomForestClassifier,        rand_for_params]
    ,["MLP",     MLPClassifier,        mlp_for_params]

]

insights = []
#  Treina e testa cada algoritmo com seus respectivos hiperparâmetros
for modelname, Model, params_list in modelclasses:
    for params in params_list:
        model = Model(**params)
        cv_scores = cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=5)
        score = round(cv_scores.mean(),2)
        recall = cross_val_score(model, X_train, y_train, scoring='recall', cv=5)
        precision = cross_val_score(model, X_train, y_train, scoring='precision', cv=5)
        meanSquaredError = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
        recall_mean = recall.mean()
        precision_mean = precision.mean()
        squared_mean = meanSquaredError.mean()
        score_stddev = round(cv_scores.std(),2)
        y_pred = cross_val_predict(model, X_train, y_train, cv=10)
        conf_mat = confusion_matrix(y_train, y_pred)
        insights.append((modelname, model, params, score, score_stddev, recall_mean, precision_mean, conf_mat))

        
insights.sort(key=lambda x:x[-2], reverse=True)

# resultados
modelnames = []
scores = []
for modelname, model, params, score, score_stddev, recall_mean, precision_mean, conf_mat in insights:
    print(modelname)
    print('\nScore: ',score, ' Desv_padr: ',score_stddev)
    print("\nRecall:", recall_mean)
    print("\nPrecision: ", precision_mean)
    print("\nMean squared error: ", squared_mean*(-1))
    print(conf_mat)
    modelnames.append(modelname)
    scores.append(score)
    
# Recurso visual
plt.figure(figsize=(16,5))
plt.bar(modelnames, scores, color='red', yerr=score_stddev)
plt.ylabel('Lucro')
plt.xlabel('Algoritmo')
plt.title('Lucro total por algoritmo treinado')
plt.axis([-1,len(modelnames),-1,1.5])

In [ ]:
! pip install xgboost


In [ ]:
colunas = X_train.columns

### III.I Aprimoramento de hiperparâmetros

In [ ]:
from sklearn.model_selection import validation_curve, RandomizedSearchCV

class HyperParameterEvaluator:
    """
    Responsible for performing a search in order to discover the set of 
    specific model configuration arguments that result in the best performance 
    of the model on a test coverage dataset.
    """
    
    # -------------------------------------------------------------------------
    #           Constructor
    # -------------------------------------------------------------------------
    def __init__(self, dataframe, regressor, predictable_column, hyperparameters):
        """
        Performs a search in order to discover the set of specific model 
        configuration arguments that result in the best performance of the model
        on a test coverage dataset.
        
        :param      dataframe: Dataset to be evaluated
        :param      regressor: Machine learning algorithm
        :param      predictable_column: Column name to be predicted
        :param      hyperparameters: Hyperparameters to be analyzed. It must be
        a dictionary, where each key is the hyperparameter name and each value
        is a list of hyperparameter values to be analyzed
        """
        self.__dataset = dataframe
        self.__regressor = regressor
        self.__train_score = 0
        self.__test_score = 0
        self.__predictable_column = predictable_column
        self.__hyperparameters = hyperparameters
        
        
    # -------------------------------------------------------------------------
    #           Methods
    # -------------------------------------------------------------------------
    def evaluate_validation_curve(self, metrics):
        for name, values in self.__hyperparameters.items():
            self.__evaluate_validation_curve(name, values, metrics)
        
    def __evaluate_validation_curve(self, param_name, param_range, metrics):
        self.__build_validation_curve(param_name, param_range, metrics)
        self.__build_validation_curve_chart(param_range, param_name)
        self.__display_current_chart()
        
    def __build_validation_curve(self, param_name, param_range, metrics):
        self.__train_score, self.__test_score = validation_curve(
                self.__regressor,
                X = self.__dataset[metrics].values, 
                y = self.__dataset[self.__predictable_column].values, 
                param_name = param_name, 
                param_range = param_range,
        )
        
    def __build_validation_curve_chart(self, param_range, param_name):
        self.__build_chart_title(param_name)
        self.__build_chart_axis()
        self.__build_chart_legend()
        self.__build_chart_training_score_data(param_range)
        self.__build_chart_cross_validation_score_data(param_range)
        
    def __build_chart_title(self, param_name):
        plt.title("Validation Curve - " + param_name)
        
    def __build_chart_axis(self):
        plt.xlabel(r"$\gamma$")
        plt.ylabel("Score")
        plt.ylim(0.0, 1.1)
        
    def __build_chart_legend(self):
        plt.legend(loc="best")
        
    def __build_chart_training_score_data(self, param_range):
        train_scores_mean = np.mean(self.__train_score, axis=1)
        train_scores_std = np.std(self.__train_score, axis=1)
        test_scores_mean = np.mean(self.__test_score, axis=1)
        test_scores_std = np.std(self.__test_score, axis=1)
        
        plt.semilogx(param_range, train_scores_mean, label="Training score",
                     color="darkorange", lw=2)
        plt.fill_between(param_range, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.2,
                         color="darkorange", lw=2)
        
    def __build_chart_cross_validation_score_data(self, param_range):
        train_scores_mean = np.mean(self.__train_score, axis=1)
        train_scores_std = np.std(self.__train_score, axis=1)
        test_scores_mean = np.mean(self.__test_score, axis=1)
        test_scores_std = np.std(self.__test_score, axis=1)
        
        plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
                     color="navy", lw=2)
        plt.fill_between(param_range, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.2,
                         color="navy", lw=2)
        
    def __display_current_chart(self):
        plt.show()
        
    def evaluate_hyperparam(self, metrics, hyperparam_space):
        hyperparam_table = self.__build_hyperparam_table(metrics, hyperparam_space)
        self.__display_hyperparam_table(hyperparam_table)
        
    def __build_hyperparam_table(self, metrics, hyperparam_space):
        hyperparam_table = RandomizedSearchCV(
            estimator=self.__regressor, 
            param_distributions=hyperparam_space,
            n_iter=100,
            random_state=0
        )

        hyperparam_table.fit(self.__dataset[metrics].values, self.__dataset[self.__predictable_column].values)
        
        return hyperparam_table
        
    def __display_hyperparam_table(self, hyperparam_table):
        display(hyperparam_table.best_params_)

In [ ]:
mlp_hyperparams = {
    "activation": ["identity", "logistic", "tanh", "relu"],
    "solver": ["lbfgs", "sgd", "adam"],
    "alpha": [x for x in np.linspace(1e-4, 1e4, 10, dtype=float)],
    "tol": [x for x in np.linspace(1e-4, 1e4, 10, dtype=float)],
}
metrics = df_depressed_dataset.columns.values[:-1]
hpe = HyperParameterEvaluator(
    dataframe = df_depressed_dataset,
    regressor = MLPClassifier(random_state=0), 
    predictable_column = "depressed", 
    hyperparameters = mlp_hyperparams
)

hpe.evaluate_validation_curve(metrics)

## iv) interpretação do modelo treinado, 
buscando obter insights sobre o impacto dos atributos na tomada de decisão